In [1]:
import lxml
import cchardet
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import requests, re, html

In [84]:
df_big = pd.DataFrame(columns=['ID','Title','IngredientsRaw','TotalTime','NumSteps', 
                           'AvgRating','NumReviews', 'RecipeURL'])

In [85]:
# %%time

def normalize_string(string):
    # Convert all named and numeric character references (e.g. &gt;, &#62;)
    unescaped_string = html.unescape(string)
    return re.sub(
        r"\s+", " ",
        unescaped_string.replace("\xa0", " ")
        .replace("\n", " ")  # &nbsp;
        .replace("\t", " ").strip(),)

def getrecipe(url, retry_count=0):
    sleep(0.01 + 0.02*np.random.random())
    
    df = pd.DataFrame(columns=['ID','Title','IngredientsRaw','TotalTime','NumSteps', 
                           'AvgRating','NumReviews', 'RecipeURL'])
    df.loc[0,'RecipeURL'] = url
    
    try: 
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
    except ConectionResetError as e:
        if retry_count == 5:
            raise e
        time.sleep(1 + 2*np.random.random())
        getrecipe(url, retry_count + 1)        
    
    if soup.find("h1").get_text() != 'Whoops...':
        
        df.loc[0,'Title'] =  soup.find("h1").get_text()
            
        try:
            NumReviews =  soup.find("a", {"class": "reviews-count__link theme-color"}).getText()
            df.loc[0,'NumReviews'] = eval(NumReviews)
        except:
            pass


        try:
            ingredients = soup.findAll("li", {"class": "recipe-ingredients__item"})
            df.loc[0,'IngredientsRaw'] = [normalize_string(ingredient.get_text()) + '#item' 
                                          for ingredient in ingredients]

        except:
            pass

        try:
             df.loc[0,'TotalTime'] = (soup.find("div", {"class": "recipe-facts__time"})
                                         .get_text().replace('READY IN: ',''))
        except:
            pass

        try:
            instructions = soup.findAll("li", {"class": "recipe-directions__step"})
            df.loc[0,'NumSteps'] = len(instructions)
        except:
            pass

        try:
            AvgRating = soup.find('div',class_='stars-rate__filler').get("style")[:-2].split(':')[1]
            df.loc[0,'AvgRating'] = float(AvgRating)/20
        except:
            pass
        
    return df
    
for i in range(23561,40000):
    url  = 'https://www.food.com/recipe/' + str(i) 
    df = getrecipe(url,5)
    df.loc[0,'ID'] = i
    df_big = df_big.append(df)
    
df_big.sample(20)

NameError: name 'ConectionResetError' is not defined

In [87]:
# df_big4 = df_big.copy()
df_big4

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,23561,Avocado with Salmon Appetizer,"[2 avocados, peeled & quartered#item, 8 sliced...",10mins,4,5.0,1,https://www.food.com/recipe/23561
0,23562,Shrimp or Crawfish Monica,"[1⁄2 cup butter#item, 1 lb shrimp or 1 lb cray...",30mins,5,4.0,3,https://www.food.com/recipe/23562
0,23563,Asparagus and cheese puffs,"[1 tablespoon oil (I prefer olive oil)#item, 1...",25mins,11,NaN,NaN,https://www.food.com/recipe/23563
0,23564,Good Morning Bridgeford Bread Rolls,"[1 package bridgford bread roll#item, 1 packag...",8hrs 30mins,5,NaN,NaN,https://www.food.com/recipe/23564
0,23565,Perfect Pizza Dough,"[1 cup water#item, 1 teaspoon sugar#item, 1 (1...",2hrs,12,4.4,5,https://www.food.com/recipe/23565
...,...,...,...,...,...,...,...,...
0,26281,Steak Sauce Extraordinaire,[1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb fro...,1hr 20mins,4,5.0,2,https://www.food.com/recipe/26281
0,26282,Beef Barbecue Biscuits,"[3⁄4 lb ground beef#item, 1⁄2 cup barbecue sau...",25mins,4,3.0,1,https://www.food.com/recipe/26282
0,26283,A-1 Dumplings,"[1 cup flour#item, 1⁄2 teaspoon salt#item, 2 t...",33mins,6,5.0,7,https://www.food.com/recipe/26283
0,26284,Quick & Easy Fried Rice,"[beef or shrimp, sliced thinly (whatever you h...",20mins,17,4.56,9,https://www.food.com/recipe/26284


In [73]:
# df_big3 = df_big.copy()
df_big3

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,18114,Three Bean Soup,"[1 lb ground beef#item, 1⁄4 cup olive oil#item...",35mins,5,4.33,4,https://www.food.com/recipe/18114
0,18115,Honey Glazed Acorn Squash,"[4 cups acorn squash, peeled and cubed#item, 1...",1hr 5mins,6,5.0,6,https://www.food.com/recipe/18115
0,18116,Chocolate Eclair Cake,"[1 (1 lb) box honey graham crackers#item, FILL...",20mins,12,4.97,31,https://www.food.com/recipe/18116
0,18117,Chicken in Mead Sauce,[4 chicken legs-thighs (wings and/or thighs)#i...,1hr 20mins,7,4.5,4,https://www.food.com/recipe/18117
0,18118,Scottish Crowdie,[20 fluid ounces whole milk (that has just bec...,6hrs,8,NaN,NaN,https://www.food.com/recipe/18118
...,...,...,...,...,...,...,...,...
0,23556,Crock Pot Hamburger Soup,"[24 ounces lean ground beef#item, 1 sweet onio...",9hrs 20mins,6,4.33,6,https://www.food.com/recipe/23556
0,23557,Broccoli Cashew Rice Pilaf,"[1⁄4 cup butter (Don't substitute)#item, 2 oun...",40mins,9,5.0,3,https://www.food.com/recipe/23557
0,23558,Mediterranean Veggie Salad,[4 lemon cucumbers or 4 pickling cucumbers#ite...,3hrs 20mins,17,5.0,1,https://www.food.com/recipe/23558
0,23559,Cherry Chip Cornflake Cookies,"[2 1⁄4 cups flour#item, 1⁄8 teaspoon salt#item...",42mins,7,5.0,5,https://www.food.com/recipe/23559


In [67]:
# df_big2 = df_big.copy()
df_big2

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,12619,Balsamic Asparagus,[1 bunch fresh asparagus (stalks approx same t...,10mins,5,4.63,95,https://www.food.com/recipe/12619
0,12620,Halloween Green Bread by Barbaja,"[1⁄4 cup shortening#item, 2 cups cornmeal#item...",35mins,4,NaN,NaN,https://www.food.com/recipe/12620
0,12621,Alternative to Peanut Butter,"[1 cup tahini#item, 2⁄3 cup molasses#item]",12mins,1,3.0,4,https://www.food.com/recipe/12621
0,12622,Jalapeno-Olive Salsa,"[3 tomatoes, seeded and coarsely chopped#item,...",15mins,4,5.0,1,https://www.food.com/recipe/12622
0,12623,Frozen Bread Dough,[12 -13 cups bread flour (If I use some whole ...,1hr 40mins,12,5.0,2,https://www.food.com/recipe/12623
...,...,...,...,...,...,...,...,...
0,18109,Mo's Red Salad Dressing,"[2 cups ketchup#item, 1 1⁄2 cups sugar#item, 2...",10mins,10,4.0,1,https://www.food.com/recipe/18109
0,18110,Winner's Spaghetti Casserole,"[1 (10 -16 ounce) package thin spaghetti, cook...",55mins,14,4.52,30,https://www.food.com/recipe/18110
0,18111,Caramel Apples In Rum Sauce,"[3⁄4 cup butter#item, 1 1⁄3 cups sugar#item, 1...",18mins,7,5.0,1,https://www.food.com/recipe/18111
0,18112,Fresh Orange Pound Cake with Orange Glaze,"[8 ounces butter, room temperature#item, 1 cup...",45mins,18,2.0,1,https://www.food.com/recipe/18112


In [68]:
# df_big1 = df_big.copy()
df_big1

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"[48 large tomatoes#item, 2 teaspoons salt#item]",3hrs 15mins,8,3.0,2,https://www.food.com/recipe/10000
0,10001,Aloo Dum,"[1⁄2 kg potato (small and round)#item, oil (fo...",4mins,18,NaN,NaN,https://www.food.com/recipe/10001
0,10002,Caulinana salad,"[1 small cauliflower#item, 2 bananas#item, 2 t...",15mins,11,NaN,NaN,https://www.food.com/recipe/10002
0,10003,Spicy Corn Salad With Avocado Dressing,"[4 cups whole kernel corn (I use frozen)#item,...",12mins,10,3.67,3,https://www.food.com/recipe/10003
0,10004,Ginger Beer,"[TO MAKE THE GINGER BEER#item, 1 1⁄2 teaspoons...",312hrs 5mins,18,4.0,2,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
0,12614,Fat Free Tomato Sauce,"[1 (800 g) can Italian tomatoes#item, 1 stalk ...",50mins,7,5.0,1,https://www.food.com/recipe/12614
0,12615,Fruit Smoothie Delight,"[2 ripe nectarines or 8 strawberries#item, 1 b...",1min,2,NaN,NaN,https://www.food.com/recipe/12615
0,12616,Texas Green Fried Tomatoes,"[1⁄2 cup flour#item, 1⁄2 cup yellow cornmeal#i...",25mins,6,5.0,6,https://www.food.com/recipe/12616
0,12617,Swedish Lutefisk,"[1 piece dried lutefisk, sawed into 6 inch len...",480hrs 10mins,10,4.67,7,https://www.food.com/recipe/12617


In [293]:
df.sample(10)

,ID,Title,Ingredients,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
444,444,Luscious Applesauce Bars,"[Bars, 1 cup brown sugar, firmly packed, 1⁄2 c...",1hr 46mins,14,4.0,2,https://www.food.com/recipe/444
700,700,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,1486,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,546,Green Death Tuna Sandwich,"[1 (12 ounce) can tuna, drained and shredded, ...",10mins,5,4.75,4,https://www.food.com/recipe/546
1968,1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,182,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1056,1056,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,133,Blueberry Pancake Syrup,"[4 cups blueberries, stemmed, rinsed and drain...",1hr 26mins,13,4.75,4,https://www.food.com/recipe/133
286,286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
705,705,Tortilla Peach Cobbler,"[1⁄2 cup sugar, 1 tablespoon cornstarch, 16 ou...",30mins,9,4.5,2,https://www.food.com/recipe/705


In [93]:
df_bigall = pd.concat([df_big1, df_big2, df_big3, df_big4])
df_bigall

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"[48 large tomatoes#item, 2 teaspoons salt#item]",3hrs 15mins,8,3.0,2,https://www.food.com/recipe/10000
0,10001,Aloo Dum,"[1⁄2 kg potato (small and round)#item, oil (fo...",4mins,18,NaN,NaN,https://www.food.com/recipe/10001
0,10002,Caulinana salad,"[1 small cauliflower#item, 2 bananas#item, 2 t...",15mins,11,NaN,NaN,https://www.food.com/recipe/10002
0,10003,Spicy Corn Salad With Avocado Dressing,"[4 cups whole kernel corn (I use frozen)#item,...",12mins,10,3.67,3,https://www.food.com/recipe/10003
0,10004,Ginger Beer,"[TO MAKE THE GINGER BEER#item, 1 1⁄2 teaspoons...",312hrs 5mins,18,4.0,2,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
0,26281,Steak Sauce Extraordinaire,[1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb fro...,1hr 20mins,4,5.0,2,https://www.food.com/recipe/26281
0,26282,Beef Barbecue Biscuits,"[3⁄4 lb ground beef#item, 1⁄2 cup barbecue sau...",25mins,4,3.0,1,https://www.food.com/recipe/26282
0,26283,A-1 Dumplings,"[1 cup flour#item, 1⁄2 teaspoon salt#item, 2 t...",33mins,6,5.0,7,https://www.food.com/recipe/26283
0,26284,Quick & Easy Fried Rice,"[beef or shrimp, sliced thinly (whatever you h...",20mins,17,4.56,9,https://www.food.com/recipe/26284


In [137]:
test = df_bigall.set_index('ID').sort_index()
test['IngredientsRaw'] = test['IngredientsRaw'].apply(lambda x: ', '.join(x) if np.all(pd.notnull(x)) else x)
test.to_csv('data/df26285str.csv',index=True)
df26285str = pd.read_csv("data/df26285str.csv")
df26285str

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"48 large tomatoes#item, 2 teaspoons salt#item",3hrs 15mins,8.0,3.00,2.0,https://www.food.com/recipe/10000
1,10001,Aloo Dum,"1⁄2 kg potato (small and round)#item, oil (for...",4mins,18.0,NaN,NaN,https://www.food.com/recipe/10001
2,10002,Caulinana salad,"1 small cauliflower#item, 2 bananas#item, 2 ta...",15mins,11.0,NaN,NaN,https://www.food.com/recipe/10002
3,10003,Spicy Corn Salad With Avocado Dressing,"4 cups whole kernel corn (I use frozen)#item, ...",12mins,10.0,3.67,3.0,https://www.food.com/recipe/10003
4,10004,Ginger Beer,"TO MAKE THE GINGER BEER#item, 1 1⁄2 teaspoons ...",312hrs 5mins,18.0,4.00,2.0,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
16281,26281,Steak Sauce Extraordinaire,1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb froz...,1hr 20mins,4.0,5.00,2.0,https://www.food.com/recipe/26281
16282,26282,Beef Barbecue Biscuits,"3⁄4 lb ground beef#item, 1⁄2 cup barbecue sauc...",25mins,4.0,3.00,1.0,https://www.food.com/recipe/26282
16283,26283,A-1 Dumplings,"1 cup flour#item, 1⁄2 teaspoon salt#item, 2 te...",33mins,6.0,5.00,7.0,https://www.food.com/recipe/26283
16284,26284,Quick & Easy Fried Rice,"beef or shrimp, sliced thinly (whatever you ha...",20mins,17.0,4.56,9.0,https://www.food.com/recipe/26284


In [139]:
test = df_bigall.set_index('ID').sort_index()
test.to_csv('data/df26285list.csv',index=True)
df26285list = pd.read_csv("data/df26285list.csv")
df26285list

,ID,Title,IngredientsRaw,TotalTime,NumSteps,AvgRating,NumReviews,RecipeURL
0,10000,Tomato Paste,"['48 large tomatoes#item', '2 teaspoons salt#i...",3hrs 15mins,8.0,3.00,2.0,https://www.food.com/recipe/10000
1,10001,Aloo Dum,"['1⁄2 kg potato (small and round)#item', 'oil ...",4mins,18.0,NaN,NaN,https://www.food.com/recipe/10001
2,10002,Caulinana salad,"['1 small cauliflower#item', '2 bananas#item',...",15mins,11.0,NaN,NaN,https://www.food.com/recipe/10002
3,10003,Spicy Corn Salad With Avocado Dressing,['4 cups whole kernel corn (I use frozen)#item...,12mins,10.0,3.67,3.0,https://www.food.com/recipe/10003
4,10004,Ginger Beer,"['TO MAKE THE GINGER BEER#item', '1 1⁄2 teaspo...",312hrs 5mins,18.0,4.00,2.0,https://www.food.com/recipe/10004
...,...,...,...,...,...,...,...,...
16281,26281,Steak Sauce Extraordinaire,['1 - 1 1⁄2 lb fresh rhubarb or 1 -1 1/2 lb fr...,1hr 20mins,4.0,5.00,2.0,https://www.food.com/recipe/26281
16282,26282,Beef Barbecue Biscuits,"['3⁄4 lb ground beef#item', '1⁄2 cup barbecue ...",25mins,4.0,3.00,1.0,https://www.food.com/recipe/26282
16283,26283,A-1 Dumplings,"['1 cup flour#item', '1⁄2 teaspoon salt#item',...",33mins,6.0,5.00,7.0,https://www.food.com/recipe/26283
16284,26284,Quick & Easy Fried Rice,"['beef or shrimp, sliced thinly (whatever you ...",20mins,17.0,4.56,9.0,https://www.food.com/recipe/26284
